# Selección de características (Feature selection)

¿Por qué es necesario seleccionar características? Para algunos, esta idea puede parecer contraintuitiva, pero hay al menos dos razones importantes para deshacerse de características sin importancia. La primera es evidente para cualquier ingeniero: cuantos más datos, mayor complejidad computacional. Mientras trabajemos con conjuntos de datos de juguete, el tamaño de los datos no es un problema, pero, para sistemas de producción con carga real, cientos de características adicionales serán bastante tangibles. La segunda razón es que algunos algoritmos toman el ruido (características no informativas) como una señal y sobreajustan.

### Aproximaciones estadísticas

El candidato más obvio para ser eliminado es un rasgo cuyo valor permanece invariable, es decir, que no contiene ninguna información. Si nos basamos en esta idea, es razonable afirmar que las características con una varianza baja son peores que las que tienen una varianza alta. Por lo tanto, se puede considerar la posibilidad de eliminar características con una varianza inferior a un determinado umbral.

## Basado en estadísticos no supervisado

Generamos un problema tipo para nuestras pruebas.

In [2]:
from sklearn.datasets import make_classification

x_data_generated, y_data_generated = make_classification(random_state=2)
x_data_generated.shape

(100, 20)

## VarianceThreshold

- **¿Qué es?**: Una técnica que elimina características cuya varianza no alcanza un umbral especificado.
- **Funcionamiento**: Calcula la varianza de cada característica y elimina aquellas que no superan el umbral establecido. Esto significa que las características que apenas cambian entre muestras (es decir, tienen baja varianza) se descartarán.
- **Ventajas**: Rápida y fácil de usar, útil cuando las características con baja varianza no son informativas.
- **Limitaciones**: No considera la relación con la variable objetivo, puede eliminar características útiles.


In [3]:
from sklearn.feature_selection import VarianceThreshold

primer_filtro = VarianceThreshold(.7).fit_transform(x_data_generated)
primer_filtro.shape

(100, 20)

In [4]:
primer_filtro = VarianceThreshold(.8).fit_transform(x_data_generated)
primer_filtro.shape

(100, 15)

In [5]:
primer_filtro = VarianceThreshold(.9).fit_transform(x_data_generated)
primer_filtro.shape

(100, 14)

In [6]:
primer_filtro = VarianceThreshold(.99).fit_transform(x_data_generated)
primer_filtro.shape

(100, 11)

#### Basado en estadísticos supervisado

## SelectKBest

- **¿Qué es?**: Selecciona las K mejores características basadas en algún criterio de puntuación.
- **Funcionamiento**: Calcula un puntaje para cada característica en función de algún criterio (como la prueba F para la regresión lineal o la puntuación chi-cuadrado para la clasificación). Luego, selecciona las K características con los puntajes más altos.
- **Ventajas**: Permite seleccionar características en función de su relevancia con respecto a la variable objetivo.
- **Limitaciones**: Puede ser costoso para conjuntos de datos grandes, elección subjetiva de K.


In [7]:
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

x_data_kbest = SelectKBest(k=5).fit_transform(x_data_generated, y_data_generated)

In [8]:
x_data_kbest.shape

(100, 5)

In [9]:
import numpy as np

logit = LogisticRegression(random_state=42)
coss_val_res = cross_val_score(logit, x_data_generated, y_data_generated, cv=5)
np.mean(coss_val_res)

0.8800000000000001

In [10]:
coss_val_res = cross_val_score(logit, x_data_kbest, y_data_generated, cv=5)
np.mean(coss_val_res)

0.86

In [11]:
x_data_varth = VarianceThreshold(.9).fit_transform(x_data_generated)
coss_val_res = cross_val_score(logit, x_data_varth, y_data_generated, cv=5)
np.mean(coss_val_res)

0.89

Podemos ver que las características seleccionadas han mejorado la calidad del clasificador. Por supuesto, este ejemplo es puramente artificial; sin embargo, merece la pena utilizarlo para problemas reales.

## Basado en modelos supervisados

### RFE (Recursive Feature Elimination)

- **¿Qué es?**: Elimina recursivamente las características menos importantes hasta alcanzar el número deseado.
- **Funcionamiento**: Entrena un modelo (generalmente un modelo de aprendizaje automático) en todas las características y luego elimina las características con menor importancia. Este proceso se repite hasta que se alcanza el número deseado de características.
- **Ventajas**: Considera la interacción entre características, puede ser más robusto.
- **Limitaciones**: Puede ser costoso computacionalmente, elección del modelo y número de características.


In [12]:
from sklearn.svm import SVC
from sklearn.feature_selection import RFE

svc = SVC(kernel="linear", C=1)
rfe = RFE(estimator=svc, n_features_to_select=10, step=1)
rfe.fit(x_data_generated, y_data_generated)

RFE(estimator=SVC(C=1, kernel='linear'), n_features_to_select=10)

In [13]:
x_rfe = rfe.transform(x_data_generated)
x_rfe.shape

(100, 10)

In [14]:
coss_val_res = cross_val_score(logit, x_rfe, y_data_generated, cv=5)
np.mean(coss_val_res)

0.8700000000000001

In [15]:
coss_val_res = cross_val_score(svc, x_rfe, y_data_generated, cv=5)
np.mean(coss_val_res)

0.9099999999999999

In [16]:
x_data_varth = VarianceThreshold(.9).fit_transform(x_data_generated)
coss_val_res = cross_val_score(svc, x_data_varth, y_data_generated, cv=5)
np.mean(coss_val_res)

0.9

In [17]:
coss_val_res = cross_val_score(svc, x_data_generated, y_data_generated, cv=5)
np.mean(coss_val_res)

0.85